### PostInstalls
Do these first by navigating to [POSTINSTALLS](http://ec2-54-210-238-172.compute-1.amazonaws.com/notebooks/Notebooks/PostInstall/PostInstalls.ipynb)

Then come on back! and continue with the next cell in this notebook
##### Author: dana
##### email: dana.e.moore@leidos.com

In [9]:
#import packages
import pandas as pd 
import numpy as np
import nltk
import re

import hdbscan
import umap
import contractions

from bertopic import BERTopic
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer

from dynamoDButils import *
from flair.embeddings import TransformerDocumentEmbeddings

import boto3
import logging
import uuid

import pickle

import time
from datetime import date, datetime 
import datetime
from datetime import timedelta

from os import listdir
import os.path as osp
from os.path import isfile, join

#LOCALS
RESOURCE = 'dynamodb'
TABLE = "Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev"
REGION = "us-east-1"

CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"
MODELPATH= "/home/deleidos/Notebooks/Models"
def complete(state="complete"):
#     print(f"Now: {datetime.datetime.now()}")
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")
complete(state="imports done")


Cell imports done @ 2023-01-09 20:58:39


In [2]:
# DynamoDb Connection
ddb = boto3.resource(RESOURCE)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
# Fetch all the records in DynamoDb

articles = extendedScanArticlesNoKey(RESOURCE,TABLE,columns="title,summary")
print (f"Resp length: {len(articles[0])}", end="\n\n")
print (f"Resp[6000] : {articles[6000]}")


page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 4
extendedScanArticlesNoKey: Length: 7647 
Resp length: 2

Resp[6000] : {'summary': {'S': '[1/5] Chinese President Xi Jinping waves after his speech as the new Politburo Standing Committee members meet the media following the 20th National Congress of the Communist Party of China, at the Great Hall of the People in Beijing, China October 23, 2022. REUTERS/Tingshu Wang/File PhotoBEIJING, Dec 29 (Reuters) - Xi Jinping secured an historic third leadership term in October, emerging as China\'s most powerful ruler since Mao Zedong, bolstered by a Politburo Standing Committee stacked with allies and no successor-in-waiting to challenge him.It was a rare highlight for Xi in 2022, a tumultuous year capped by unpr

In [3]:
# # Replace getTitles function on Utils for ES
# titles = []
# for i in articles['Items']:
#     titles.append({'label' : i['title'], 'value' : i['url']})
# for t in titles:
#     print (t['label'])
# complete(state="titles")
titles = []
summaries=[]

for article in articles:
    titles.append (article['title']['S'])
    summaries.append(article['summary']['S'])

print (f"first title  : {titles[0]}")
print (f"first summary: {summaries[0]}")


first title  : Cisco partners with Microsoft to add Teams to its meeting devices
first summary: Networking firm Cisco Systems Inc. will add Microsoft Corp.'s Teams messaging app to its meeting devices, the two firms said on Wednesday, offering users an alternative to its own Webex video conferencing app.Ticker Security Last Change Change % CSCO CISCO SYSTEMS INC. 39.27 -0.31 -0.78%Ticker Security Last Change Change % MSFT MICROSOFT CORP. 225.75 +0.34 +0.15%Cisco's Jeetu Patel, head of Security & Collaboration, said the company aims to be the hardware platform for a wide range of conferencing software platforms.MELINDA GATES OPENS UP ON ‘UNBELIEVABLY PAINFUL’ DIVORCE"There's going to be times that people want to jump on a Microsoft Teams call, they want to jump on a Zoom call, they want to jump on a Google call."Ilya Bukshteyn, vice president of Microsoft Teams Calling and Devices, told Reuters the Teams Room software already runs on several other hardware devices and will be available 

In [4]:
df = pd.DataFrame(summaries, columns =['text'])

# remove protocols
df.text = df.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(), 1)

# remove non-letters
df.text = df.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)

# convert to lowercase
df.text = df.text.apply(lambda x: ' '.join([w.lower() for w in x.split()]))

# expand contractions  
df.text = df.text.apply(lambda x: ' '.join([contractions.fix(w) for w in x.split()]))

# remove stopwords
stop_words = stopwords.words('english')
df.text = df.text.apply(lambda x: ' '.join([w for w in x.split() if w not in stop_words]))

# remove short words
df.text = df.text.apply(lambda x: ' '.join([w.strip() for w in x.split() if len(w.strip()) >= 3]))

# lemmatize
df.text = df.text.apply(lambda x: ' '.join([WordNetLemmatizer().lemmatize(w) for w in x.split()]))

docs = df.text.to_list()

print (f"Number of articles for training the model: {len(docs)}")    

complete(state="models")

Number of articles for training the model: 7647

Cell models @ 2023-01-09 19:59:44


In [10]:
%%time
# create model

# # Load sentence transformer model
# sentence_model = SentenceTransformer("all-distilroberta-v1")

# # Create documents embeddings
embeddings = sentence_model.encode(docs, show_progress_bar=True)
# sentence_model = TransformerDocumentEmbeddings('roberta-large')
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

# Define UMAP model to reduce embeddings dimension
# umap_model = umap.UMAP(n_neighbors=9,
#                        n_components=5,
#                        min_dist=0.0,
#                        metric='cosine',
#                        low_memory=False)
# Set the random state in the UMAP model to prevent stochastic behavior 
umap_model = umap.UMAP(n_neighbors=15
                  , n_components=5
                  , min_dist=0.0
                  , metric='cosine'
                  , random_state=42
                  , low_memory=False)

# Define HDBSCAN model to perform documents clustering
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=3,
                                min_samples=1,
                                metric='euclidean',
                                cluster_selection_method='eom',
                                prediction_data=True)

# Create BERTopic model
topic_model = BERTopic(embedding_model=sentence_model,
                       nr_topics=1,
                       top_n_words=8,
                       n_gram_range=(1,3), 
                       calculate_probabilities=True,
                       umap_model= umap_model,
                       hdbscan_model=hdbscan_model,
                       verbose=True)

# Train model, extract topics and probabilities
topics, probabilities = topic_model.fit_transform(docs)
# Save and load model
modelfile = join(MODELPATH, "BERTopicV2.mdl")
topic_model.save(modelfile)
my_model = BERTopic.load(modelfile)
new_topics, new_probs = my_model.transform(docs, embeddings)

topicspickle = join(MODELPATH, "BERTopicV2.pickle")

pickle.dump(topics, open( topicspickle, "wb" ) )
assert topics == new_topics
complete(state="topics, probabilities done. Save & load checked")

2023-01-09 20:59:51,315 - BERTopic - Transformed documents to Embeddings


2023-01-09 21:00:16,413 - BERTopic - Reduced dimensionality
2023-01-09 21:08:09,424 - BERTopic - Clustered reduced embeddings
2023-01-09 21:08:54,220 - BERTopic - Reduced number of topics from 942 to 2
2023-01-09 21:09:21,215 - BERTopic - Reduced dimensionality
2023-01-09 21:17:17,116 - BERTopic - Calculated probabilities with HDBSCAN
2023-01-09 21:17:17,117 - BERTopic - Predicted clusters



Cell topics, probabilities done. Save & load checked @ 2023-01-09 21:17:17
CPU times: user 18min 15s, sys: 14.2 s, total: 18min 29s
Wall time: 18min 12s


In [14]:
topic_model.visualize_barchart(top_n_topics=30)

In [16]:
# get docs from topic

df = pd.DataFrame({'topic': topics, 'document': docs})
docs_in_topic = df[df.topic == 0]
docs_in_topic


,topic,document
1,0,apple ceo tim cook confirmed apple buy made mi...
7,0,santa clara california based company reported ...
8,0,although iphone sale strong analyst targeted s...
12,0,article msftgoogl follow favorite stock create...
65,0,nov reuters chip designer computing firm nvidi...
...,...,...
7581,0,sign microsoft corp company office central bus...
7595,0,nasdaq stock future fell premarket wednesday a...
7596,0,sign posted front oracle headquarters december...
7614,0,salesforce crm laid cost cutting plan wednesda...


In [17]:
info = topic_model.get_topic_info()
print (f"topic_model info= \n{info}")

topic_model info= 
   Topic  Count                       Name
0     -1   7323   -1_data_said_new_company
1      0    324  0_stock_company_said_year


In [ ]:
tpcs = []
for t in info["Topic"][0:int(len(info))]: 
    tpcarray = topic_model.get_topic(t)
    for elt in tpcarray:
        tpcs.append(elt[0])
tpcs = list (set(tpcs))
tpcs.sort()
print (tpcs)

In [ ]:
# ddb = boto3.client('dynamodb')
# def addTopic(topic_name, upvote, downvote, index_name):
    
#     topic = {}

#     topic["topic"] = {
#         "S": topic_name
#     }
    
#     topic["upvote"] = {
#         "N": upvote
#     }
    
#     topic["downvote"] = {
#         "N": upvote
#     }

#     return ddb.put_item(
#             TableName=index_name,
#             Item=topic
#             )

In [ ]:
# for topic in tpcs:    
#     ret = addTopic(topic, upvote='1', downvote='0', index_name='InnovationBlog_Article_Topics')
#     if ret:
#         print (f"document {topic} created")
#     else:
#         print (f"failed to create document {topic}")


In [ ]:
# Heat Maps, Rankings, Barcharts